In [1]:
## Load the library: 

from torch.utils.data import Dataset, DataLoader
from model_creation_torch import *
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

2023-04-20 08:45:29.451135: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-20 08:45:29.559088: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-20 08:45:29.934906: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-20 08:45:29.934941: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

In [2]:
# Create the data:


!python make_data_spd2spd.py  10000 5 20 10
!python make_data_spd2spd.py  10000 20 20 10
!python make_data_spd2spd.py  10000 5 40 20
!python make_data_spd2spd.py  10000 20 40 20


python: can't open file '/home/hieunguyen/Desktop/MlSupervise/Experiment/experiment/make_data_spd2spd.py': [Errno 2] No such file or directory
python: can't open file '/home/hieunguyen/Desktop/MlSupervise/Experiment/experiment/make_data_spd2spd.py': [Errno 2] No such file or directory
python: can't open file '/home/hieunguyen/Desktop/MlSupervise/Experiment/experiment/make_data_spd2spd.py': [Errno 2] No such file or directory
python: can't open file '/home/hieunguyen/Desktop/MlSupervise/Experiment/experiment/make_data_spd2spd.py': [Errno 2] No such file or directory


In [2]:
# Delete old models for retrained

!rm trained_models/*
!rm training_history/*

# Hyperparameters

ns = [10000, 1000, 100]
contexts = ['10000_20_10_5_0', '10000_20_10_20_0', '10000_40_20_5_0', '10000_40_20_20_0']
batch_size = 32
epochs = 30
lr = 0.001
skip_training = False

# early_stop = EarlyStopping(monitor='val_loss', patience=3)

# Custom Dataset

class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = self.X[idx]
        y = self.Y[idx]
        return x, y
    
# Training and testing loop:

def train_loop(dataloader, model, loss_fn, optimizer):
    num_batches = len(dataloader)
    size = len(dataloader.dataset)
    train_loss = 0.0

    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        with torch.no_grad():
            train_loss += loss.item()
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    return train_loss / num_batches


def test_loop(dataloader, model, loss_fn):
    num_batches = len(dataloader)
    test_loss=  0.0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

    test_loss /= num_batches
    print(f"Test Error: \n  Avg loss: {test_loss:>8f} \n")
    return test_loss

# Utils function for saving:

def create_history_df():
    history = pd.DataFrame(columns= ['Epochs', 'TrainLoss', 'TestLoss'])
    return history

def add_record_to_dataframe(df, record):
    # Convert the record tuple to a DataFrame with a single row
    new_row = pd.DataFrame([record], columns=df.columns)
    
    # Concatenate the new DataFrame with the original DataFrame
    df = pd.concat([df, new_row], ignore_index=True)
    
    return df

def save_dataframe_to_pickle(df, n, context, name):
    filepath = f'training_history/{name}_{context}_{n}.pickle'
    df.to_pickle(filepath)
    return filepath
    

def save_model(model, n, context, name):
    filepath = f'trained_models/{name}_{context}_{n}.pt'
    torch.save(model.state_dict(), filepath)
    return filepath

# def save_history_df(history, n, context, name):

# Training results

task = []
nof_data = []
dir_of_trained_modes = []
dir_of_training_history = []

# Loop

if not skip_training:
    for n in ns:
        for context in contexts:
            rank = int(context.split('_')[3])
            X = np.load('data/spd2spd/X_'+context+'.npy')
            Y = np.load('data/spd2spd/Y_'+context+'.npy')
            Xt = np.load('data/spd2spd/Xt_'+context+'.npy')
            Yt = np.load('data/spd2spd/Yt_'+context+'.npy')
    
            # Create training dataset and dataloader
            train_dataset = CustomDataset(X[0:n], Y[0:n])
            train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            
            # Create test dataset and dataloader
            test_dataset = CustomDataset(Xt, Yt)
            test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
            # Multivariate Linear Regression:
            print("MLR ---------------------------------------------")
            history = create_history_df()
            model = MVl(X[0].shape, Y[0].shape)
            loss_fn = nn.MSELoss()
            optimizer = torch.optim.Adam(model.parameters(), lr=lr)
            
            
            for t in range(epochs):
                print(f"Epoch {t+1}\n-------------------------------")
                train_loss = train_loop(train_dataloader, model, loss_fn, optimizer)
                test_loss = test_loop(test_dataloader, model, loss_fn)
                history = add_record_to_dataframe(history, (t, train_loss, test_loss))
    
            dir_of_trained_modes.append(save_model(model, n, context, "MVL"))
            dir_of_training_history.append(save_dataframe_to_pickle(history, n, context, "MVL"))
            task.append(context)
            nof_data.append(n)
    
            # Partial Trace Regression:
            print("PTR ---------------------------------------------")
            history = create_history_df()
            output_shape = Y.shape[1]
            input_shape = X[0].shape
            model = KrausLayer(input_shape, output_shape, rank)
            loss_fn = nn.MSELoss()
            optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
            for t in range(epochs):
                print(f"Epoch {t+1}\n-------------------------------")
                train_loss = train_loop(train_dataloader, model, loss_fn, optimizer)
                test_loss = test_loop(test_dataloader, model, loss_fn)
                history = add_record_to_dataframe(history, (t, train_loss, test_loss))
    
            dir_of_trained_modes.append(save_model(model, n, context, "PTR"))
            dir_of_training_history.append(save_dataframe_to_pickle(history, n, context, "PTR"))
            task.append(context)
            nof_data.append(n)
    
    
            # Reduced Rank Regression
            # 
            history = create_history_df()    
            model = RRMVL(rank * 10, X[0].shape, Y[0].shape)
            loss_fn = nn.MSELoss() 
            optimizer = torch.optim.Adam(model.parameters(), lr=lr)
            
            
            for t in range(epochs):
                print(f"Epoch {t+1}\n-------------------------------")
                train_loss = train_loop(train_dataloader, model, loss_fn, optimizer)
                test_loss = test_loop(test_dataloader, model, loss_fn)
                history = add_record_to_dataframe(history, (t, train_loss, test_loss))
            dir_of_trained_modes.append(save_model(model, n, context, "RRR"))
            dir_of_training_history.append(save_dataframe_to_pickle(history, n, context, "RRR"))
            task.append(context)
            nof_data.append(n)
    
            # Trace Regression
            history = create_history_df()
            model = TraceLayer(X[0].shape, Y[0].shape)
            loss_fn = nn.MSELoss() 
            optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
            for t in range(epochs):
                print(f"Epoch {t+1}\n-------------------------------")
                train_loss = train_loop(train_dataloader, model, loss_fn, optimizer)
                test_loss = test_loop(test_dataloader, model, loss_fn)
                history = add_record_to_dataframe(history, (t, train_loss, test_loss))
            dir_of_trained_modes.append(save_model(model, n, context, "TR"))
            dir_of_training_history.append(save_dataframe_to_pickle(history, n, context, "TR"))
            task.append(context)
            nof_data.append(n)


MLR ---------------------------------------------
Epoch 1
-------------------------------
loss: 1949.769531  [   32/10000]
loss: 847.845520  [ 3232/10000]
loss: 381.060913  [ 6432/10000]
loss: 174.854614  [ 9632/10000]
Test Error: 
  Avg loss: 147.197841 

Epoch 2
-------------------------------
loss: 156.910019  [   32/10000]
loss: 70.730721  [ 3232/10000]
loss: 36.338036  [ 6432/10000]
loss: 25.079008  [ 9632/10000]
Test Error: 
  Avg loss: 23.794551 

Epoch 3
-------------------------------
loss: 24.044706  [   32/10000]
loss: 18.064627  [ 3232/10000]
loss: 16.007248  [ 6432/10000]
loss: 13.088305  [ 9632/10000]
Test Error: 
  Avg loss: 12.438760 

Epoch 4
-------------------------------
loss: 11.362957  [   32/10000]
loss: 10.122611  [ 3232/10000]
loss: 8.399564  [ 6432/10000]
loss: 7.301431  [ 9632/10000]
Test Error: 
  Avg loss: 7.337085 

Epoch 5
-------------------------------
loss: 7.065270  [   32/10000]
loss: 5.468819  [ 3232/10000]
loss: 5.041320  [ 6432/10000]
loss: 3.8032

In [7]:
records = pd.read_pickle('records.pickle')


,task,nof_data,dir_of_trained_models,dir_of_training_history
0,10000_20_10_5_0,10000,trained_models/MVL_10000_20_10_5_0_10000.pt,training_history/MVL_10000_20_10_5_0_10000.pickle
1,10000_20_10_5_0,10000,trained_models/PTR_10000_20_10_5_0_10000.pt,training_history/PTR_10000_20_10_5_0_10000.pickle
2,10000_20_10_5_0,10000,trained_models/RRR_10000_20_10_5_0_10000.pt,training_history/RRR_10000_20_10_5_0_10000.pickle
3,10000_20_10_5_0,10000,trained_models/TR_10000_20_10_5_0_10000.pt,training_history/TR_10000_20_10_5_0_10000.pickle
16,10000_20_10_5_0,1000,trained_models/MVL_10000_20_10_5_0_1000.pt,training_history/MVL_10000_20_10_5_0_1000.pickle
17,10000_20_10_5_0,1000,trained_models/PTR_10000_20_10_5_0_1000.pt,training_history/PTR_10000_20_10_5_0_1000.pickle
18,10000_20_10_5_0,1000,trained_models/RRR_10000_20_10_5_0_1000.pt,training_history/RRR_10000_20_10_5_0_1000.pickle
19,10000_20_10_5_0,1000,trained_models/TR_10000_20_10_5_0_1000.pt,training_history/TR_10000_20_10_5_0_1000.pickle
32,10000_20_10_5_0,100,trained_models/MVL_10000_20_10_5_0_100.pt,training_history/MVL_10000_20_10_5_0_100.pickle
33,10000_20_10_5_0,100,trained_models/PTR_10000_20_10_5_0_100.pt,training_history/PTR_10000_20_10_5_0_100.pickle


In [31]:
history = pd.read_pickle('./training_history/TR_10000_40_20_20_0_10000.pickle')
history

,Epochs,TrainLoss,TestLoss
0,0,122841.458441,88300.934814
1,1,63603.440682,43463.740967
2,2,29580.851357,18988.123535
3,3,12091.496198,7271.216919
4,4,4339.040638,2523.921295
5,5,1474.926649,935.139261
6,6,610.431482,488.732369
7,7,384.911843,363.194283
8,8,313.900890,307.719258
9,9,270.747208,265.874728


In [40]:
pickle_files = [f for f in os.listdir("training_history") if f.endswith(".pickle")]
pickle_files

['MVL_10000_40_20_20_0_10000.pickle',
 'PTR_10000_40_20_5_0_100.pickle',
 'TR_10000_20_10_20_0_10000.pickle',
 'RRR_10000_20_10_20_0_100.pickle',
 'TR_10000_40_20_20_0_1000.pickle',
 'RRR_10000_40_20_5_0_10000.pickle',
 'TR_10000_40_20_20_0_10000.pickle',
 'PTR_10000_40_20_20_0_10000.pickle',
 'RRR_10000_40_20_5_0_1000.pickle',
 'MVL_10000_40_20_20_0_100.pickle',
 'MVL_10000_20_10_5_0_100.pickle',
 'MVL_10000_20_10_5_0_1000.pickle',
 'MVL_10000_20_10_20_0_100.pickle',
 'MVL_10000_20_10_20_0_1000.pickle',
 'RRR_10000_20_10_5_0_10000.pickle',
 'MVL_10000_40_20_20_0_1000.pickle',
 'TR_10000_40_20_5_0_100.pickle',
 'MVL_10000_40_20_5_0_10000.pickle',
 'PTR_10000_40_20_20_0_100.pickle',
 'RRR_10000_20_10_20_0_10000.pickle',
 'RRR_10000_20_10_5_0_1000.pickle',
 'TR_10000_20_10_5_0_100.pickle',
 'TR_10000_20_10_5_0_10000.pickle',
 'TR_10000_20_10_5_0_1000.pickle',
 'RRR_10000_20_10_20_0_1000.pickle',
 'PTR_10000_20_10_20_0_1000.pickle',
 'RRR_10000_20_10_5_0_100.pickle',
 'RRR_10000_40_20_20_

In [44]:
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# Define the path to the folder containing the pickle files
folder_path = "./training_history/"

# Get a list of all the pickle files in the folder
pickle_files = [f for f in os.listdir(folder_path) if f.endswith(".pickle")]

# Loop to allow the user to select multiple files to display
while True:
    # Ask the user to select a file to display
    print("Select a file to display:")
    for i, filename in enumerate(pickle_files):
        print(f"{i+1}: {filename}")
    print("Type 'Exit' to quit.")
    user_input = input()
    
    # Exit the loop if the user enters "Exit"
    if user_input == "Exit":
        break
    
    try:
        file_index = int(user_input) - 1
        selected_file = pickle_files[file_index]
        # Read the selected pickle file into a DataFrame
        filepath = os.path.join(folder_path, selected_file)
        df = pd.read_pickle(filepath)

        # Create a figure object
        fig = go.Figure()

        # Add "TrainLoss" and "TestLoss" as two separate lines to the figure
        fig.add_trace(go.Scatter(x=df.index, y=df["TrainLoss"], name="Train Loss"))
        fig.add_trace(go.Scatter(x=df.index, y=df["TestLoss"], name="Test Loss"))

        # Customize the layout of the figure
        fig.update_layout(title=f"Training History - {selected_file}", xaxis_title="Time", yaxis_title="Loss")

        # Show the figure in a web browser
        pio.show(fig)
    except:
        print("Invalid input. Please try again.")

Select a file to display:
1: MVL_10000_40_20_20_0_10000.pickle
2: PTR_10000_40_20_5_0_100.pickle
3: TR_10000_20_10_20_0_10000.pickle
4: RRR_10000_20_10_20_0_100.pickle
5: TR_10000_40_20_20_0_1000.pickle
6: RRR_10000_40_20_5_0_10000.pickle
7: TR_10000_40_20_20_0_10000.pickle
8: PTR_10000_40_20_20_0_10000.pickle
9: RRR_10000_40_20_5_0_1000.pickle
10: MVL_10000_40_20_20_0_100.pickle
11: MVL_10000_20_10_5_0_100.pickle
12: MVL_10000_20_10_5_0_1000.pickle
13: MVL_10000_20_10_20_0_100.pickle
14: MVL_10000_20_10_20_0_1000.pickle
15: RRR_10000_20_10_5_0_10000.pickle
16: MVL_10000_40_20_20_0_1000.pickle
17: TR_10000_40_20_5_0_100.pickle
18: MVL_10000_40_20_5_0_10000.pickle
19: PTR_10000_40_20_20_0_100.pickle
20: RRR_10000_20_10_20_0_10000.pickle
21: RRR_10000_20_10_5_0_1000.pickle
22: TR_10000_20_10_5_0_100.pickle
23: TR_10000_20_10_5_0_10000.pickle
24: TR_10000_20_10_5_0_1000.pickle
25: RRR_10000_20_10_20_0_1000.pickle
26: PTR_10000_20_10_20_0_1000.pickle
27: RRR_10000_20_10_5_0_100.pickle
28: 

Select a file to display:
1: MVL_10000_40_20_20_0_10000.pickle
2: PTR_10000_40_20_5_0_100.pickle
3: TR_10000_20_10_20_0_10000.pickle
4: RRR_10000_20_10_20_0_100.pickle
5: TR_10000_40_20_20_0_1000.pickle
6: RRR_10000_40_20_5_0_10000.pickle
7: TR_10000_40_20_20_0_10000.pickle
8: PTR_10000_40_20_20_0_10000.pickle
9: RRR_10000_40_20_5_0_1000.pickle
10: MVL_10000_40_20_20_0_100.pickle
11: MVL_10000_20_10_5_0_100.pickle
12: MVL_10000_20_10_5_0_1000.pickle
13: MVL_10000_20_10_20_0_100.pickle
14: MVL_10000_20_10_20_0_1000.pickle
15: RRR_10000_20_10_5_0_10000.pickle
16: MVL_10000_40_20_20_0_1000.pickle
17: TR_10000_40_20_5_0_100.pickle
18: MVL_10000_40_20_5_0_10000.pickle
19: PTR_10000_40_20_20_0_100.pickle
20: RRR_10000_20_10_20_0_10000.pickle
21: RRR_10000_20_10_5_0_1000.pickle
22: TR_10000_20_10_5_0_100.pickle
23: TR_10000_20_10_5_0_10000.pickle
24: TR_10000_20_10_5_0_1000.pickle
25: RRR_10000_20_10_20_0_1000.pickle
26: PTR_10000_20_10_20_0_1000.pickle
27: RRR_10000_20_10_5_0_100.pickle
28: 

Select a file to display:
1: MVL_10000_40_20_20_0_10000.pickle
2: PTR_10000_40_20_5_0_100.pickle
3: TR_10000_20_10_20_0_10000.pickle
4: RRR_10000_20_10_20_0_100.pickle
5: TR_10000_40_20_20_0_1000.pickle
6: RRR_10000_40_20_5_0_10000.pickle
7: TR_10000_40_20_20_0_10000.pickle
8: PTR_10000_40_20_20_0_10000.pickle
9: RRR_10000_40_20_5_0_1000.pickle
10: MVL_10000_40_20_20_0_100.pickle
11: MVL_10000_20_10_5_0_100.pickle
12: MVL_10000_20_10_5_0_1000.pickle
13: MVL_10000_20_10_20_0_100.pickle
14: MVL_10000_20_10_20_0_1000.pickle
15: RRR_10000_20_10_5_0_10000.pickle
16: MVL_10000_40_20_20_0_1000.pickle
17: TR_10000_40_20_5_0_100.pickle
18: MVL_10000_40_20_5_0_10000.pickle
19: PTR_10000_40_20_20_0_100.pickle
20: RRR_10000_20_10_20_0_10000.pickle
21: RRR_10000_20_10_5_0_1000.pickle
22: TR_10000_20_10_5_0_100.pickle
23: TR_10000_20_10_5_0_10000.pickle
24: TR_10000_20_10_5_0_1000.pickle
25: RRR_10000_20_10_20_0_1000.pickle
26: PTR_10000_20_10_20_0_1000.pickle
27: RRR_10000_20_10_5_0_100.pickle
28: 

Select a file to display:
1: MVL_10000_40_20_20_0_10000.pickle
2: PTR_10000_40_20_5_0_100.pickle
3: TR_10000_20_10_20_0_10000.pickle
4: RRR_10000_20_10_20_0_100.pickle
5: TR_10000_40_20_20_0_1000.pickle
6: RRR_10000_40_20_5_0_10000.pickle
7: TR_10000_40_20_20_0_10000.pickle
8: PTR_10000_40_20_20_0_10000.pickle
9: RRR_10000_40_20_5_0_1000.pickle
10: MVL_10000_40_20_20_0_100.pickle
11: MVL_10000_20_10_5_0_100.pickle
12: MVL_10000_20_10_5_0_1000.pickle
13: MVL_10000_20_10_20_0_100.pickle
14: MVL_10000_20_10_20_0_1000.pickle
15: RRR_10000_20_10_5_0_10000.pickle
16: MVL_10000_40_20_20_0_1000.pickle
17: TR_10000_40_20_5_0_100.pickle
18: MVL_10000_40_20_5_0_10000.pickle
19: PTR_10000_40_20_20_0_100.pickle
20: RRR_10000_20_10_20_0_10000.pickle
21: RRR_10000_20_10_5_0_1000.pickle
22: TR_10000_20_10_5_0_100.pickle
23: TR_10000_20_10_5_0_10000.pickle
24: TR_10000_20_10_5_0_1000.pickle
25: RRR_10000_20_10_20_0_1000.pickle
26: PTR_10000_20_10_20_0_1000.pickle
27: RRR_10000_20_10_5_0_100.pickle
28: 

## Partial race regression